## Quickstart

In [ ]:
import os
import numpy as np
from PIL import Image


In [ ]:
output_dir = "/mnt/home/ssa2206/Robot/SteerKep/data/camera_output"
pathify = lambda x: os.path.join(output_dir, x)


rgb_np = np.load(pathify('rgb_raw.npy'))
depth_np = np.load(pathify('depth_raw.npy'))

# Normalize depth values to 0-255
depth_normalized = (depth_np - depth_np.min()) / (depth_np.max() - depth_np.min()) * 255



In [ ]:
# Save RGB image
rgb_img = Image.fromarray(rgb_np)
rgb_img.save(pathify('rgb_image.png'))

# Save depth image
depth_img = Image.fromarray(depth_normalized)
depth_img.save(pathify('depth_image.png'))


# Rekep

In [2]:
import omnigibson
print(omnigibson.__version__)
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
import sys
base_dir = '/mnt/home/ssa2206/Robot/SteerKep/COMS6998-ReKep'
sys.path.append(base_dir)
print(f"Cuda available: {torch.cuda.is_available()}")
from pprint import pprint

1.1.1
Cuda available: True


In [3]:
from environment import ReKepOGEnv
from keypoint_proposal import KeypointProposer
from constraint_generation import ConstraintGenerator
from ik_solver import IKSolver
from subgoal_solver import SubgoalSolver
from path_solver import PathSolver
from visualizer import Visualizer
import transform_utils as T
from omnigibson.robots.fetch import Fetch
from utils import (
    bcolors,
    get_config,
    load_functions_from_txt,
    get_linear_interpolation_steps,
    spline_interpolate_poses,
    get_callable_grasping_cost_fn,
    print_opt_debug_dict,
)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
global_config = get_config(config_path=os.path.join(base_dir, 'configs/config.yaml'))

In [6]:
os.environ["OMNIGIBSON_HEADLESS"] = "1"
os.environ["OMNIGIBSON_DISABLE_GRAPHICS"] = "1"  # Completely disable graphics
os.environ["OMNIGIBSON_NO_RENDERER"] = "1"  # Disable renderer initialization


In [7]:
def stage1_disturbance_seq(env):
    """
    Move the pen in stage 0 when robot is trying to grasp the pen
    """
    pen = env.og_env.scene.object_registry("name", "pen_1")
    holder = env.og_env.scene.object_registry("name", "pencil_holder_1")
    # disturbance sequence
    pos0, orn0 = pen.get_position_orientation()
    pose0 = np.concatenate([pos0, orn0])
    pos1 = pos0 + np.array([-0.08, 0.0, 0.0])
    orn1 = T.quat_multiply(T.euler2quat(np.array([0, 0, np.pi/4])), orn0)
    pose1 = np.concatenate([pos1, orn1])
    pos2 = pos1 + np.array([0.10, 0.0, 0.0])
    orn2 = T.quat_multiply(T.euler2quat(np.array([0, 0, -np.pi/2])), orn1)
    pose2 = np.concatenate([pos2, orn2])
    control_points = np.array([pose0, pose1, pose2])
    pose_seq = spline_interpolate_poses(control_points, num_steps=25)
    def disturbance(counter):
        if counter < len(pose_seq):
            pose = pose_seq[counter]
            pos, orn = pose[:3], pose[3:]
            pen.set_position_orientation(pos, orn)
            counter += 1
    counter = 0
    while True:
        yield disturbance(counter)
        counter += 1

def stage2_disturbance_seq(env):
    """
    Take the pen out of the gripper in stage 1 when robot is trying to reorient the pen
    """
    apply_disturbance = env.is_grasping()
    pen = env.og_env.scene.object_registry("name", "pen_1")
    holder = env.og_env.scene.object_registry("name", "pencil_holder_1")
    # disturbance sequence
    pos0, orn0 = pen.get_position_orientation()
    pose0 = np.concatenate([pos0, orn0])
    pose1 = np.array([-0.30, -0.15, 0.71, -0.7071068, 0, 0, 0.7071068])
    control_points = np.array([pose0, pose1])
    pose_seq = spline_interpolate_poses(control_points, num_steps=25)
    def disturbance(counter):
        if apply_disturbance:
            if counter < 20:
                if counter > 15:
                    env.robot.release_grasp_immediately()  # force robot to release the pen
                else:
                    pass  # do nothing for the other steps
            elif counter < len(pose_seq) + 20:
                env.robot.release_grasp_immediately()  # force robot to release the pen
                pose = pose_seq[counter - 20]
                pos, orn = pose[:3], pose[3:]
                pen.set_position_orientation(pos, orn)
                counter += 1
    counter = 0
    while True:
        yield disturbance(counter)
        counter += 1

def stage3_disturbance_seq(env):
    """
    Move the holder in stage 2 when robot is trying to drop the pen into the holder
    """
    pen = env.og_env.scene.object_registry("name", "pen_1")
    holder = env.og_env.scene.object_registry("name", "pencil_holder_1")
    # disturbance sequence
    pos0, orn0 = holder.get_position_orientation()
    pose0 = np.concatenate([pos0, orn0])
    pos1 = pos0 + np.array([-0.02, -0.15, 0.0])
    orn1 = orn0
    pose1 = np.concatenate([pos1, orn1])
    control_points = np.array([pose0, pose1])
    pose_seq = spline_interpolate_poses(control_points, num_steps=5)
    def disturbance(counter):
        if counter < len(pose_seq):
            pose = pose_seq[counter]
            pos, orn = pose[:3], pose[3:]
            holder.set_position_orientation(pos, orn)
            counter += 1
    counter = 0
    while True:
        yield disturbance(counter)
        counter += 1


In [8]:
base_dir = '/mnt/home/ssa2206/Robot/SteerKep/COMS6998-ReKep'
task_list = {
    'pen': {
        'scene_file': os.path.join(base_dir, 'configs/og_scene_file_pen.json'),
        'instruction': 'reorient the white pen and drop it upright into the black pen holder',
        'rekep_program_dir': os.path.join(base_dir, 'vlm_query/pen'),
        'disturbance_seq': {1: stage1_disturbance_seq, 2: stage2_disturbance_seq, 3: stage3_disturbance_seq},
        },
}

In [9]:
task = task_list['pen']
scene_file = task['scene_file']
instruction = task['instruction']

In [10]:
env = ReKepOGEnv(global_config['env'], scene_file, verbose=True)

[INFO] [omnigibson.simulator] ----- Starting OmniGibson. This will take 10-30 seconds... -----


Starting kit application with the following args:  ['/mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/apps/omnigibson_4_1_0.kit', '--/app/tokens/exe-path=/mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/omni', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=False', '--/app/fastShutdown=False', '--ext-folder', '/mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/exts', '--ext-folder', '/mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/apps', '--/phys

2025-03-27 18:18:30 [7,958ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_INCOMPATIBLE_DRIVER
2025-03-27 18:18:30 [7,972ms] [Error] [carb.graphics-vulkan.plugin] vkCreateInstance failed. Vulkan 1.1 is not supported, or your driver requires an update.
2025-03-27 18:18:30 [7,973ms] [Error] [gpu.foundation.plugin] carb::graphics::createInstance failed.
2025-03-27 18:18:30 [8,265ms] [Error] [carb.graphics-vulkan.plugin] VkResult: ERROR_INCOMPATIBLE_DRIVER
2025-03-27 18:18:30 [8,265ms] [Error] [carb.graphics-vulkan.plugin] vkCreateInstance failed. Vulkan 1.1 is not supported, or your driver requires an update.
2025-03-27 18:18:30 [8,265ms] [Error] [gpu.foundation.plugin] carb::graphics::createInstance failed.
2025-03-27 18:18:30 [8,535ms] [Error] [omni.gpu_foundation_factory.plugin] Failed to create any GPU devices, including an attempt with compatibility mode.
sh: 1: zenity: not found
[INFO] [omni.kit.telemetry.impl.sentry_extension] sentry is disabled for external build
[INFO] [

[8.757s] [ext: omni.kit.pipapi-0.0.0] startup
[8.766s] [ext: omni.kit.pip_archive-0.0.0] startup
[8.769s] [ext: omni.mtlx-0.1.0] startup
[8.778s] [ext: omni.usd.config-1.0.4] startup
[8.915s] [ext: omni.gpucompute.plugins-0.0.0] startup
[8.921s] [ext: omni.usd.libs-1.0.1] startup
[9.524s] [ext: omni.kit.telemetry-0.5.0] startup
2025-03-27 18:18:31 [9,325ms] [Warning] [omni.platforminfo.plugin] failed to load XRAndR.
[9.572s] [ext: omni.kit.loop-isaac-1.2.0] startup
[9.579s] [ext: omni.kit.test-0.0.0] startup
[9.774s] [ext: omni.appwindow-1.1.8] startup
2025-03-27 18:18:31 [9,566ms] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2025-03-27 18:18:31 [9,566ms] [Warning] [carb] Failed to startup plugin carb.windowing-glfw.plugin (interfaces: [carb::windowing::IGLContext v1.0],[carb::windowing::IWindowing v1.4]) (impl: carb.windowing-glfw.plugin)
2025-03-27 18:18:31 [9,572ms] [Warning] [carb.windowing-glfw.plugin] GLFW initialization failed.
2025-03-27 18:18:31 [9,572ms]

2025-03-27 18:18:31 [9,576ms] [Error] [omni.appwindow.plugin] Failed to acquire IWindowing interface
2025-03-27 18:18:31 [9,598ms] [Error] [omni.kit.renderer.plugin] GPU Foundation is not initialized!
2025-03-27 18:18:31 [9,686ms] [Error] [omni.kit.imgui_renderer.plugin] _prepareDefaultWindow: invalid device attached to the default window!


[9.917s] [ext: omni.ui-2.23.11] startup
[9.975s] [ext: omni.kit.mainwindow-1.0.3] startup
[9.998s] [ext: carb.audio-0.1.0] startup
[10.036s] [ext: omni.uiaudio-1.0.0] startup
[10.049s] [ext: omni.kit.uiapp-0.0.0] startup
[10.049s] [ext: omni.usd.schema.audio-0.0.0] startup
[10.476s] [ext: omni.usd.schema.physx-106.0.20] startup
[10.677s] [ext: omni.usd.schema.forcefield-106.0.20] startup
[10.719s] [ext: omni.usd.schema.anim-0.0.0] startup
[10.890s] [ext: omni.usd.schema.omniscripting-1.0.0] startup
[10.941s] [ext: omni.usd.schema.omnigraph-1.0.0] startup
[11.004s] [ext: omni.anim.graph.schema-106.0.2] startup
[11.048s] [ext: omni.anim.navigation.schema-106.0.2] startup
[11.078s] [ext: omni.usd.schema.isaac-2.1.0] startup
[11.134s] [ext: omni.usd.schema.semantics-0.0.0] startup
[11.160s] [ext: omni.usd.schema.geospatial-0.0.0] startup
[11.188s] [ext: omni.usd.schema.scene.visualization-2.0.2] startup
[11.203s] [ext: omni.graph.exec-0.9.3] startup
[11.212s] [ext: omni.usd_resolver-1.0.0]

2025-03-27 18:18:34 [12,454ms] [Error] [omni.ext.plugin] Could not load the dynamic library from /mnt/home/ssa2206/miniforge3/envs/omnigibson/lib/python3.10/site-packages/isaacsim/extscache/omni.iray.libs/bin/iray/libneuray.so. Error: libGLU.so.1: cannot open shared object file: No such file or directory (note that this may be caused by a dependent library)


: 

: 

: 

In [ ]:
main = Main(scene_file, visualize=args.visualize)
main.perform_task(instruction,
                rekep_program_dir=task['rekep_program_dir'] if args.use_cached_query else None,
                disturbance_seq=task.get('disturbance_seq', None) if args.apply_disturbance else None)